In [8]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from tqdm.notebook import tqdm

class BacteriaGrowthModel():
  def __init__(self,
               size: int,
               initial_bacteria: float,
               initial_food: float,
               p_add_food: float,
               food_diff_rate: float,
               bacteria_diff_rate: float,
               bacteria_growth_rate: float,
               food_growth_rate: float,
               consumption_rate: float,
               max_food_capacity: float = 100):

    # initial parameters
    self.size = size
    self.current_state = np.zeros((size, size, 2)) # square matrix where each cell contains a list of two numbers
    self.next_state = np.zeros((size, size, 2))

    self.initial_food = initial_food
    self.initial_bacteria = initial_bacteria

    # constraint - hardcoded to 100 as per requirements
    self.max_food_capacity = 100
    self.p_add_food = p_add_food

    # diffusion rates
    self.food_diff_rate = food_diff_rate
    self.bacteria_diff_rate = bacteria_diff_rate

    # growth rates
    self.bacteria_growth_rate = bacteria_growth_rate
    self.food_growth_rate = food_growth_rate

    # consumption rate
    self.consumption_rate = consumption_rate

    # step counter for visualization
    self.step_counter = 0

  def initialize(self):
    # Initialize with food in every cell
    self.current_state[:, :, 1] = self.initial_food

    # Place a single bacterium in the middle of the grid
    center = self.size // 2
    self.current_state[center, center, 0] = self.initial_bacteria

    self.figure, self.axes = plt.subplots(1, 2, figsize=(12, 5))

  def draw(self):
    # Plot bacteria
    plot1 = self.axes[0].imshow(
        self.current_state[:, :, 0], vmin=0, vmax=1, cmap='Blues')
    self.axes[0].set_title(f'Bacteria at step {self.step_counter}')

    # Plot food
    plot2 = self.axes[1].imshow(
        self.current_state[:, :, 1], vmin=0, vmax=self.max_food_capacity, cmap='Greens')
    self.axes[1].set_title(f'Food at step {self.step_counter}')

    return [plot1, plot2]

  def grow_food(self):
    self.next_state = np.copy(self.current_state)
    food_indices = np.where(self.current_state[:, :, 1] < self.max_food_capacity)
    food_values = self.current_state[food_indices[0], food_indices[1], 1]
    growth_factor = 1 + self.food_growth_rate * (1 - food_values / self.max_food_capacity)
    self.next_state[food_indices[0], food_indices[1], 1] = food_values * growth_factor
    self.current_state, self.next_state = self.next_state, self.current_state
    self.step_counter += 1

  def reseed_food(self):
    self.next_state = np.copy(self.current_state)
    # Use fixed probability of 0.01 as per requirements
    mask = np.random.uniform(0, 1, (self.size, self.size)) < self.p_add_food
    self.next_state[:, :, 1][mask] += 1
    self.current_state, self.next_state = self.next_state, self.current_state
    self.step_counter += 1

  def diffuse_food(self):
    self.next_state = np.copy(self.current_state)
    for row in range(self.size):
      for col in range(self.size):
        removed_food = self.food_diff_rate * self.current_state[row, col, 1]
        self.next_state[row, col, 1] -= removed_food
        for dx, dy in [(0,1), (0,-1), (1,0), (-1,0)]:
          self.next_state[(row + dx) % self.size, (col + dy) % self.size, 1] += (1/4) * removed_food
    self.current_state, self.next_state = self.next_state, self.current_state
    self.step_counter += 1

  def consume_food(self):
    self.next_state = np.copy(self.current_state)
    for row in range(self.size):
      for col in range(self.size):
        required_food = self.consumption_rate * self.current_state[row, col, 0]
        food_consumed = np.minimum(self.current_state[row, col, 1], required_food)
        bacteria_survived = food_consumed / self.consumption_rate
        # Store the bacteria that survived
        self.next_state[row, col, 0] = bacteria_survived
        self.next_state[row, col, 1] -= food_consumed
    self.current_state, self.next_state = self.next_state, self.current_state
    self.step_counter += 1

  def starvation(self):
    # Starvation is already handled in the consume_food method
    # where bacteria_survived = food_consumed / self.consumption_rate
    pass

  def reproduce_bacteria(self):
    self.next_state = np.copy(self.current_state)
    for row in range(self.size):
      for col in range(self.size):
        # Bacteria that survived (got food) reproduce according to the growth rate
        self.next_state[row, col, 0] = self.current_state[row, col, 0] * (1 + self.bacteria_growth_rate)
    self.current_state, self.next_state = self.next_state, self.current_state
    self.step_counter += 1

  def diffuse_bacteria(self):
    self.next_state = np.copy(self.current_state)
    for row in range(self.size):
      for col in range(self.size):
        removed_bacteria = self.bacteria_diff_rate * self.current_state[row, col, 0]
        self.next_state[row, col, 0] -= removed_bacteria
        for dx, dy in [(0,1), (0,-1), (1,0), (-1,0)]:
          self.next_state[(row + dx) % self.size, (col + dy) % self.size, 0] += (1/4) * removed_bacteria
    self.current_state, self.next_state = self.next_state, self.current_state
    self.step_counter += 1

def update(frame, sim, steps_per_frame, progress_bar):
    for _ in range(steps_per_frame):
        # Execute steps in the required order
        sim.grow_food()
        sim.reseed_food()
        sim.diffuse_food()
        sim.consume_food()
        sim.starvation()  # This is a no-op but included for process clarity
        sim.reproduce_bacteria()
        sim.diffuse_bacteria()
    plots = sim.draw()
    progress_bar.update(1)
    return plots

def make_animation(sim, total_frames, steps_per_frame, interval=100):
    frame_number = 0
    sim.initialize()
    progress_bar = tqdm(total=total_frames)
    update(frame_number, sim, steps_per_frame, progress_bar)
    animation = FuncAnimation(
        sim.figure,
        update,
        fargs=(sim, steps_per_frame, progress_bar),
        init_func=lambda: [],
        frames=total_frames,
        interval=interval,
        blit=True
    )
    output = HTML(animation.to_html5_video())
    sim.figure.clf()
    plt.close(sim.figure)
    return output

model = BacteriaGrowthModel(
size=100,
initial_bacteria=1.0,
initial_food=50.0,
p_add_food=0.01,
food_diff_rate=0.5,
bacteria_diff_rate=0.1,
bacteria_growth_rate=0.2,
food_growth_rate=0.1,
consumption_rate=0.1,
max_food_capacity=100
)
animation = make_animation(model, total_frames=100, steps_per_frame=10, interval=100)

  0%|          | 0/100 [00:00<?, ?it/s]

In [9]:
animation